In [11]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Digital_Music_5.json.gz')
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5.0,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5.0,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5.0,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5.0,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4.0,4.5; music to dream to,1200528000,"01 17, 2008"


In [12]:
df = df.dropna(axis=0,how = 'any')
df.info

<bound method DataFrame.info of            reviewerID        asin                 reviewerName helpful  \
0      A3EBHHCZO6V2A4  5555991584         Amaranth "music fan"  [3, 3]   
1       AZPWAXJG9OJXV  5555991584                    bethtexas  [0, 0]   
2      A38IRL0X2T4DPF  5555991584                  bob turnley  [2, 2]   
3      A22IK3I6U76GX0  5555991584                        Calle  [1, 1]   
4      A1AISPOIIHTHXX  5555991584                  Cloud "..."  [1, 1]   
...               ...         ...                          ...     ...   
64701  A1PQ1PESSO8CMO  B00KILDVEI             Ginger Christmas  [0, 0]   
64702  A120RH58WVY4W6  B00KILDVEI  Kelly Dunwell "avid reader"  [0, 0]   
64703  A19VJ2IQLO50G0  B00KILDVEI                      melinda  [0, 1]   
64704   AUDSM2CTLLW1Q  B00KILDVEI           Patrick L. Randall  [0, 0]   
64705  A1GN8UJIZLCA59  B00KILDVEI                     P Magnum  [1, 2]   

                                              reviewText  overall  \
0      It'

In [13]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df, test_size=0.20,random_state=0)
train.shape,test.shape
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [14]:
import numpy as np
import math
train['reviewText'][5]
a,b = train['reviewerID'][:2]
a_list,b_list = train[train['reviewerID']==a],train[train['reviewerID']==b]
#x = [i for i in a_list['overall']]
a_mean = np.mean([i for i in a_list['overall']]) 
b_mean = np.mean([i for i in b_list['overall']]) 
ab_merge = pd.merge(a_list,b_list,on=['asin'],how='inner')
period =  len(ab_merge) / math.sqrt(len(a_list)*len(b_list))
period

0.0199363055707225

In [15]:
reviewers,products = [],[]
reviewers.extend(i for i in train['reviewerID'] if i not in reviewers)
print(len(reviewers)) #reviewer 数量
products.extend(i for i in train['asin'] if i not in products)
print(len(products)) #product 数量

5541
3568


In [16]:
def user_similarity(train):
    item_users = dict()
    
    for user,item in zip(train['reviewerID'],train['asin']):
        if item not in item_users:
            item_users[item] = set()
        item_users[item].add(user)
        
    #稀疏矩阵C[u][v]= Nu Nv () ()
    N = dict() #用户商品数量
    C = {}     #稀疏矩阵
    
    for item,user in item_users.items():
        for u in user:
            if u not in N:
                N[u] =0
            N[u] += 1
            for v in user:
                if u == v:
                    continue
                if u not in C:
                    C[u]={}
                if v not in  C[u]:
                    C[u][v]=0
                C[u][v]+=1
    #相似矩阵W
    W = {}
    for u,related_users in C.items():
         for v, cuv in related_users.items():
                
                if u not in W:
                    W[u]={}
                if v not in  W[u]:
                    W[u][v]=0
                W[u][v] = cuv / math.sqrt(N[u] * N[v]) 
    
    return W    
    
    
W = user_similarity(train)

In [17]:
from operator import itemgetter
def recommend (user,train,W,K):
    rank = {}
    interacted = train[train['reviewerID']==user]
    interacted_items = [i for i in interacted['asin']]
    for v, wuv in sorted(W[user].items(), key=itemgetter(1), reverse=True)[0:K]:   #选前K个相关用户
        for i in train[train['reviewerID']==v]['asin']: #用户V用过的产品中
            if i in interacted_items: 
                    continue
            if i not in rank:
                rank[i] = 0
            rank[i] += wuv * 1 
    return rank

[i for i in recommend('A2XT5OR3AR1PU2',train,W,5).keys()][0:10]

['B000051Y37',
 'B00000G6HL',
 'B000002KBB',
 'B000000ORB',
 'B007EM6FTW',
 'B0000025WQ',
 'B00000K5AI',
 'B00000269M',
 'B000002OPL',
 'B000002L7G']

In [18]:
## 平均值预测
def PredictAll(records):
    total,predict = {},{}
    for user,vote in zip(train['reviewerID'],train['overall']):
        if vote == 0.0:
            continue
        if user not in total:
            total[user] = set()
        total[user].add(vote)
    
    for i in total.keys(): 
        average = np.mean([i for i in total[i]])
        if i not in predict:
            predict[i] = set()
        predict[i] = average 
    return predict

predict = PredictAll(train)

In [19]:
# RMSE
def RMSE(test,predict):
    total = {}
    for user,vote in zip(test['reviewerID'],test['overall']):
        if user not in total:
            total[user] = set()
        if vote != 0.0:
            total[user].add(vote)
    
    sqrt = []
    for i in total.keys():
        if i not in predict :
            continue
        for vote in total[i]:
            sqrt.append(pow(vote-predict[i],2))
           
    #print(sqrt)
    final = sum(sqrt)
    print(final,len(sqrt),len(total.keys()))
    return math.sqrt(final/len(sqrt))

RMSE(test,predict)

9834.937499999958 6989 4496


1.18625561725505

In [20]:
import collections
#d = collections.defaultdict(lambda :10)

ave_vote = dict() #用户平均打分字典
    
for user,vote in zip(train['reviewerID'],train['overall']):
    if user not in ave_vote:
        ave_vote[user]=set()
    ave_vote[user].add(vote)
for i in ave_vote.keys():
    ave_vote[i] = np.mean([t for t in ave_vote[i]])

def UserSimilarity(train,ave_vote):
    item_users = dict() #item 对应user打分的字典
    
    for user,item,vote in zip(train['reviewerID'],train['asin'],train['overall']):
        if item not in item_users:
            item_users[item] = dict()
        if user not in item_users[item]:
            item_users[item][user] = set()
            
        item_users[item][user].add(vote)
        
    
    
    
    
    #print(ave_vote[i])
    nu = dict() # 用户和平均评分
    W = dict() #相似度矩阵
    
    for i,ri in item_users.items(): #item 和item对应的用户
        for u,rui in ri.items():  # user 和他们打的分
            
            if u not in nu:   #用户u和他的评分方差
                nu[u] = 0.0
            
            for votes in rui:
                nu[u] += (votes - ave_vote[u])*(votes - ave_vote[u])
            

            for v,rvi in ri.items(): #用户v和u的评分差的乘积
                if u == v :
                    continue
                if u not in W:
                    W[u] = dict()
                    if v not in W[u]:
                        W[u][v]= 0.0
                else:
                    continue
                #print(v,rvi,rui,type(rui))


                W[u][v] += abs((list(rui)[0] - ave_vote[u])*(list(rvi)[0] - ave_vote[v]))
                #W[u][v] += (list(rui)[0] - ave_vote[u])*(list(rvi)[0] - ave_vote[v])
                #addToMat(W, u, v, \ (rui - ave_vote[u])*(rvi - ave_vote[v])) 
    for u in W:
        W[u] = {x:y/math.sqrt(nu[x]*nu[u]) for x,y in W[u].items()}
                
    return W
                



user_similarity = UserSimilarity(train,ave_vote)

<ipython-input-20-fbeff1bf68a8>:58: RuntimeWarning: invalid value encountered in double_scalars
  W[u] = {x:y/math.sqrt(nu[x]*nu[u]) for x,y in W[u].items()}


In [31]:
[i for i in user_similarity.items()][0:10]
sorted(W['A2QOI0S9Y457VW'].items(),  key=itemgetter(1), reverse=True)[0:10]

[('A2T33U7BJ2517Z', 0.5393598899705937),
 ('A2F7UCDYPS7YBA', 0.4923659639173309),
 ('AL67ODRO833W9', 0.4558423058385518),
 ('A26BH1PFS1PKVS', 0.4558423058385518),
 ('A27MNKTRN5H7WF', 0.45226701686664544),
 ('A2M2SIFK6F3IDK', 0.45226701686664544),
 ('A1JS1BXDWD3CJI', 0.40451991747794525),
 ('A39MMTSRCSPCYZ', 0.40451991747794525),
 ('A3E2PIF7OHZEAX', 0.40451991747794525),
 ('AYDVB1OR3ILA5', 0.40451991747794525)]

In [25]:
def PredictAll(train, test, ave_vote, user_similarity, K):
    users_item = dict() #item 对应user打分的字典
    
    for user,item,vote in zip(train['reviewerID'],train['asin'],train['overall']):
        if user not in users_item:
            users_item[user] = dict()
        if item not in users_item[user]:
            users_item[user][item] = set()
            
        users_item[user][item].add(vote)
        
    for user,item,vote in zip(test['reviewerID'],test['asin'],test['overall']):
        predict = 0.0 #预测值
        norm = 0.0 
        lists = sorted(user_similarity[user].items(),  key=itemgetter(1), reverse=True)[0:K]
        #print(lists[0:5])
        for v,wuv in lists:
            
            
            if item in users_item[v]:
                rvi = users_item[v][item]
                #print(rvi)
                predict += wuv * (list(rvi)[0] - ave_vote[v]) 
                norm += abs(wuv) 
        if norm > 0:
            predict /= norm
        predict += ave_vote[user]
        test.loc[test['asin']==item,'predict'] = predict
    return test

final_test = PredictAll(train, test, ave_vote, user_similarity, 5)
final_test[0:10]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,predict
0,A8IFUOL8S9BZC,B000NA27TE,IRate,"[1, 3]",3 1/2Though occasionally showing a nasty skele...,3.0,Collect this part,1270166400,"04 2, 2010",4.500000
1,A1DAFWS98LQD2R,B00000163G,"marriediimuzik ""ilovehousemuzik.""","[1, 1]",I really don't know what to say about this rec...,5.0,CLASSIC.,1308960000,"06 25, 2011",5.000000
2,A38TSW0CGV982H,B000BM6AVA,"Charles Passantino ""Andy""","[0, 2]","I just bought this album today, with Mezmerize...",5.0,"Not bad, not bad at all",1133654400,"12 4, 2005",3.500000
3,A2JV8PH4NGQPJ6,B000AL730O,"Sir George Martini ""Verbalosity""","[15, 17]",Paul McCartney never lost his talent for writi...,5.0,McCartney's Most Consistent Release,1128556800,"10 6, 2005",4.000000
4,A3JULKQLVLWZV,B000000WCT,old_hyperbolic_squiggly,"[1, 8]",I really enjoy 6 Underground and Spin Spin Sug...,2.0,"so-so, disappointing, swanky, dry on music",984528000,"03 14, 2001",3.333333
5,A1HIOSHOIZ03XP,B005F6NA56,Kayuga,"[0, 0]","Not surprisingly, the lack of an itunes commer...",5.0,Its excellent!!!!!!!! so of course no one is l...,1322611200,"11 30, 2011",4.000000
6,AWPT31A3YL590,B0000025RI,Dee Smalls!,"[0, 1]",I love this album! I have this album a long ti...,5.0,MIchael Jackson! Love the album! Rest in peace!,1324944000,"12 27, 2011",3.000000
7,A3UTAF0L13PTBW,B0001XARY0,Guylaine Le Ber,"[0, 1]",It's a very cool album! It has more in commun...,5.0,Right on kick drum,1119657600,"06 25, 2005",2.500000
8,A2PV6GK1HV54Y9,B00008OWZE,ADRIENNE MILLER,"[2, 5]",Jewel's 0304 is a drastic change from the folk...,4.0,"""Just give in to this sweet temptation...""",1145491200,"04 20, 2006",4.500000
9,A2QOI0S9Y457VW,B00004Z41Q,"Clew ""Clew""","[3, 4]","Yes, she's another pop queen. No, she's not l...",5.0,Something very different,1005350400,"11 10, 2001",3.000000


In [26]:
def RMSE_US(test):
    sqrt = 0
    all = 0
    for user,vote,predict in zip(test['reviewerID'],test['overall'],test['predict']):
        #print(vote,predict)
        if math.isnan(predict):
            continue
        else:
            all+=1
        sqrt += pow((vote-predict),2)
        #print(sqrt,pow((vote-predict),2))
    
    print(sqrt,len(test),all)
    return math.sqrt(sqrt/all)

RMSE_US(final_test)

21848.354166666566 12906 12757


1.308684883169165